In [1]:
import psycopg2
from psycopg2 import sql
from pprint import pprint as pp
from collections import defaultdict
import string
import itertools
import copy
from math import log1p
from queue import deque
import ast
import gc
from queue import deque

import nltk 
#nltk.download('wordnet')
#nltk.download('omw')
#nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn

stw_set = set(stopwords.words('english')) - {'will'}

# Connect to an existing database
conn = psycopg2.connect("dbname=imdb-movieinfo user=postgres")

# Open a cursor to perform database operations
cur = conn.cursor()

# para word embeddings
import gensim.models.keyedvectors as word2vec

In [2]:
import gensim.models.keyedvectors as word2vec

model = word2vec.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin"
    , binary=True, limit=300000)

In [3]:
embeddingHash = {}

In [4]:
cur.execute(
        "SELECT distinct table_name, column_name FROM INFORMATION_SCHEMA.columns WHERE table_schema!='pg_catalog' AND table_schema !='information_schema';"
    )
for row in cur.fetchall():
    table_name = row[0]
    column_name = row[1]
    embeddingHash.setdefault(table_name,{})
    embeddingHash[table_name][column_name]={}
    
pp(embeddingHash)


{'casting': {'__search_id': {},
             'id': {},
             'movie_id': {},
             'note': {},
             'nr_order': {},
             'person_id': {},
             'person_role_id': {},
             'role_id': {}},
 'character': {'__search_id': {},
               'id': {},
               'imdb_id': {},
               'imdb_index': {},
               'name': {},
               'name_pcode_nf': {},
               'surname_pcode': {}},
 'movie': {'__search_id': {},
           'episode_nr': {},
           'episode_of_id': {},
           'id': {},
           'imdb_id': {},
           'imdb_index': {},
           'kind_id': {},
           'phonetic_code': {},
           'production_year': {},
           'season_nr': {},
           'series_years': {},
           'title': {}},
 'person': {'__search_id': {},
            'id': {},
            'imdb_id': {},
            'imdb_index': {},
            'name': {},
            'name_pcode_cf': {},
            'name_pcode_nf': {},
   

In [5]:
embeddingHash = {
    'casting': {
        'note': {},
    },
    'character': {
        'name': {},
    },
    'movie': {
        #'episode_nr': {},
        #'production_year': {},
        #'season_nr': {},
        #'series_years': {},
        'title': {}
    },
    'person': {
        'name': {},
    },
    'quotes': {
        'info': {},
        'note': {}
    },
    'role': {
        'role': {}
    }
}

In [8]:
embeddingsA={
    'casting': set(),
    'character': set(),
    'info': set(),
    'movie': set(),
    'name': set(),
    'note': set(),
    'person': set(),
    'quotes': set(),
    'role': set(),
    'title': set()
}

for key in embeddingsA.keys():
    for (word,sim) in model.most_similar(positive=key):
        embeddingsA[key].add(word.lower().strip(string.punctuation))
        
pp(embeddingsA)

{'casting': {'actors',
             'audition',
             'auditioning',
             'auditions',
             'cast',
             'casted',
             'casting',
             'casts',
             'recasting'},
 'character': {'backstory',
               'baddie',
               'characters',
               'persona',
               'personality',
               'plotline',
               'protagonist',
               'storyline',
               'titular_character',
               'villain'},
 'info': {'com',
          'deets',
          'details',
          'infomation',
          'information',
          'log_onto',
          'please_visit_www.samsung.com',
          'please_visit_www.starwoodhotels.com',
          'tidbits',
          'www'},
 'movie': {'cinema',
           'film',
           'films',
           'flick',
           'horror_flick',
           'movie',
           'moviegoers',
           'movies',
           'romantic_comedy',
           'sequel'},
 'name': {'m

In [9]:
embeddingsB={
    'casting': set(),
    'character': set(),
    'info': set(),
    'movie': set(),
    'name': set(),
    'note': set(),
    'person': set(),
    'quotes': set(),
    'role': set(),
    'title': set(),
    
    ('casting','note'): set(),
    ('character', 'name'): set(),
    ('movie', 'title'): set(),
    ('person', 'name'): set(),
    ('quotes', 'info'): set(),
    ('quotes', 'note'): set(),
    ('role', 'role'): set(),
    
    ('person', 'casting'): set(),
    ('role', 'casting'): set(),
    ('character', 'casting'): set(),
    ('movie', 'casting'): set(),
    ('movie','quotes'): set(),    
}


for key in embeddingsB.keys():
    for (word,sim) in model.most_similar(positive=key):
        embeddingsB[key].add(word.lower().strip(string.punctuation))
        
pp(embeddingsB)

{'casting': {'actors',
             'audition',
             'auditioning',
             'auditions',
             'cast',
             'casted',
             'casting',
             'casts',
             'recasting'},
 'character': {'backstory',
               'baddie',
               'characters',
               'persona',
               'personality',
               'plotline',
               'protagonist',
               'storyline',
               'titular_character',
               'villain'},
 'info': {'com',
          'deets',
          'details',
          'infomation',
          'information',
          'log_onto',
          'please_visit_www.samsung.com',
          'please_visit_www.starwoodhotels.com',
          'tidbits',
          'www'},
 'movie': {'cinema',
           'film',
           'films',
           'flick',
           'horror_flick',
           'movie',
           'moviegoers',
           'movies',
           'romantic_comedy',
           'sequel'},
 'name': {'m